In [1]:
import requests
import datetime

In [2]:
with open('reddit_credentials.txt', 'r', encoding="utf8") as f:
    credentials = f.read()
    
credentials = credentials.split('\n')

In [3]:
# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth(credentials[0], credentials[1])

In [4]:
# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': credentials[2],
        'password': credentials[3]}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

## Get all Reddit Posts

In [5]:
all_posts = requests.get('https://api.pushshift.io/reddit/search/submission/?subreddit=singapore&sort=desc&sort_type=created_utc&size=50')

for i in range(len(all_posts.json()['data'])):
    post = all_posts.json()['data'][i]
    print('Post ID:', post['id'], 
          'Time:', datetime.datetime.fromtimestamp(post['created_utc']))
    print(post['title'], '\n')

Post ID: rry2e2 Time: 2021-12-30 17:50:31
Autopsy performed on boy who died 75 days after Covid-19 jab; public urged not to speculate on death 

Post ID: rry11n Time: 2021-12-30 17:48:08
Autopsy performed on boy who died 75 days after Covid-19 jab; public urged not to speculate on death 

Post ID: rrxq0q Time: 2021-12-30 17:29:51
Lazada refunded but item was delivered... 

Post ID: rrx504 Time: 2021-12-30 16:54:29
Mailing from Singapore to oversea 

Post ID: rrw4i6 Time: 2021-12-30 15:50:21
Singapore plans to launch country-wide facial recognition system that will replace photo IDs by 2022 

Post ID: rrw1vm Time: 2021-12-30 15:45:36
Pro-tip: do not use recycled paper to print your safe entry code and have it pasted at the entrance of the mall 

Post ID: rrvw05 Time: 2021-12-30 15:35:44
Terrible experience with shopee 

Post ID: rrvrqm Time: 2021-12-30 15:28:38
How am I supposed to connect to wifi when it’s blocked? This is my school Chromebook btw 

Post ID: rrvkmy Time: 2021-12-30 15:

## Get Comments of Individual Posts

In [6]:
# while the token is valid (~2 hours) we just add headers=headers to our requests
res = requests.get('https://oauth.reddit.com/r/singapore/comments/rmqevj', headers=headers)

# res = requests.get('https://oauth.reddit.com/r/singapore/comments/r08nub', headers=headers)

In [7]:
comments = res.json()[1]['data']['children']
limit = 3

for i in range(len(comments)):
    try:
        print('User:', comments[i]['data']['author'], 
              'Time:', datetime.datetime.fromtimestamp(comments[i]['data']['created_utc']))
        print(comments[i]['data']['body'], '\n')

        if comments[i]['data']['replies'] != '':
            replies = comments[i]['data']['replies']['data']['children']

            for j in range(len(replies)):
                print('User:', replies[j]['data']['author'], 
                      'Time:', datetime.datetime.fromtimestamp(replies[j]['data']['created_utc']))
                print(replies[j]['data']['body'], '\n')
                
        limit -= 1
        if limit == 0:
            break

    except:
        continue

User: MapleViolet Time: 2021-12-24 08:55:24
All I know is - anyone trying to pull a fast one and lie in parliament from now on will think trice,  and that's a good thing for all of us. 

User: applescript16 Time: 2021-12-23 16:24:16
Here’s some perspective: 

1) The public nature and the fact that parliamentarians have to be the one doing this of course puts up lots of visible air time but that does not imply other priorities are forsaken - just like in the private sector there are always people doing the real work while leaders are there for direction and final decisions

2) Isn’t there merit in investigating the circumstances surrounding the lies given that if indeed others were complicit in it that it would be a breach of parliamentary privilege? 

3) If you would like them to stop, regardless of your political views, would you be okay if those complicit are let off? 

User: iluj13 Time: 2021-12-23 17:01:54
Well said. It’s only a problem if your party is being investigated. If it wa